In [ ]:
# Создаем фоновый рисунок и оси
fig, ax = plt.subplots()
plt.close() # не показывать статичный фон

def animate(i):
    """
    Функция для рисования кадров.
    """
    ax.clear()
    ax.set_ylim([0, 1])
    ax.grid(True, linestyle='-.')
    ax.tick_params(labelcolor='black', labelsize='medium', width=3)
    ax.set_xlabel("x, arb. units")
    ax.set_ylabel("u(x, t), arb. units")
    ax.text(0.05, 0.9, 
            't = ' + str(np.around(T[i], decimals=3)), 
                         transform=ax.transAxes, 
                         animated=True)
            
    line = ax.plot(X, u_analytical[i], color='r', animated=True)
    return line


ani = animation.FuncAnimation(fig, animate, frames=N_t, interval=40, blit=True)
HTML(ani.to_jshtml())

In [ ]:
t_min = 0
t_max = 1
N_t = 1001

x_min = 0
x_max = 1
N_x = 200

v = 0.1

T = np.linspace(t_min, t_max, num=N_t)
X = np.linspace(x_min, x_max, num=N_x)


delta_x = X[1] - X[0]
delta_t = T[1] - T[0]
c = delta_x/delta_t


# Lattice speeds / weights
NL = 3
idxs = np.arange(NL)
cxs = c*np.array([0, 1, -1])

weights = np.array([2/4, 1/4, 1/4])
cs_squared = np.sum(weights*cxs**2)

t_relax = 0.5 + v/(delta_t*cs_squared)

shift_indexes = np.array([0, 1, -1])


def f_0_eq(u):
    return u - (u**3)/(3*c**2) - u/3

def f_1_eq(u):
    return 0.5*((u**2)/(2*c) + (u**3)/(3*c**2) + u/3)

def f_2_eq(u):
    return 0.5*(-(u**2)/(2*c) + (u**3)/(3*c**2) + u/3)


# Initial Conditions
f = np.zeros((N_x, NL))
u0 = np.sin(np.pi*X)
f[:, 0] = u0 - u0**3 /(3*c**2) - u0/3
f[:, 1] = 0.5*(u0**2 /(2*c) + u0**3 /(3*c**2) + u0/3)
f[:, 2] = 0.5*(-u0**2 /(2*c) + u0**3 /(3*c**2) + u0/3)
 
result = []
    
# Simulation Main Loop
for i in range(N_t):
    
    # Calculate fluid variables
    rho = np.sum(f, 1)
    result.append(rho)

    # Apply Collision
    feq = np.zeros(f.shape)
    feq[:, 0] = f_0_eq(rho)
    feq[:, 1] = f_1_eq(rho)
    feq[:, 2] = f_2_eq(rho)
        
    f_hat = -(1.0/t_relax)*(f - feq) + f
        
    # Drift
    for i, cx in zip(idxs, cxs):
        f[:, i] = np.roll(f_hat[:,i], shift_indexes[i], axis=0)
        
    # Apply boundary 
    f[0, :] = 0
    f[-1, :] = 0
    

In [ ]:
def TDMA(a,b,c,f):
    """
    Метод прогонки.
    """
    n = len(f)
    alpha = [0]
    beta = [0]
    x = [0] * n

    for i in range(n-1):
        alpha.append(-b[i]/(a[i]*alpha[i] + c[i]))
        beta.append((f[i] - a[i]*beta[i])/(a[i]*alpha[i] + c[i]))
            
    x[n-1] = (f[n-1] - a[n-2]*beta[n-1])/(c[n-1] + a[n-2]*alpha[n-1])

    for i in reversed(range(n-1)):
        x[i] = alpha[i+1]*x[i+1] + beta[i+1]
    
    return np.array(x)

In [ ]:
a = np.array([3,3])
b = np.array([2,1])
c = np.array([6,5,8])
f = np.array([10,16,30])
z = TDMA(a,b,c,f)
print(z)

In [ ]:
print(np.linalg.solve(np.array([[6, 2, 0],
                                [3, 5, 1],
                                [0, 3, 8]]), np.array([10,16,30])))